In [1]:
using Knet
import Base: length, size, iterate, eltype, IteratorSize, IteratorEltype, haslength, @propagate_inbounds, repeat, rand, tail
import .Iterators: cycle, Cycle, take, repeat
using Plots; default(fmt=:png,ls=:auto)

In [2]:
f = open("glove.42B.300d.txt")

IOStream(<file glove.42B.300d.txt>)

In [3]:
lines = readlines(f);

In [4]:
d1 = Knet.save("lstmn8609.jld2")
d1

#19 (generic function with 1 method)

In [5]:
ddd1["lstm"]

UndefVarError: UndefVarError: ddd1 not defined

In [6]:
embeddingdict = Dict()
embeddingindex = 1
for line in lines
    strword = split(line)
    strname = strword[1]
    embeddingdict[strname] = embeddingindex
    embeddingindex+=1
end

PREPARE DATA

In [7]:
txt1 = open("sentences.txt")
lines1 = readlines(txt1)
txt2 = open("labels.txt")
lines2 = readlines(txt2);

In [8]:
all_sentences = (x -> split(x)).(lines1);
all_sentences = [(x -> lowercase(x)).(s) for s in all_sentences];

In [9]:
train_sentences = all_sentences[1:8534]
test_sentences = all_sentences[8535:10744]
val_sentences = all_sentences[10745:11844];

In [10]:
train_labels = lines2[1:8534]
test_labels = lines2[8535:10744]
val_labels = lines2[10745:11844]
train_labels = (x -> parse(Float32, x)).(train_labels);

In [11]:
function mapfloat(label)
    if (0 <= label < 0.5); return 1; end;
    if (0.5 < label <= 1.0); return 2; end;
end
function mapstring(p)
    if p == "neg" || p == "very neg"; return 1; end;
    if p == "pos" || p == "very pos"; return 2; end;
end

mapstring (generic function with 1 method)

In [12]:
train_sentences2 = []
train_labels2 = []
test_sentences2 = []
test_labels2 = []
val_sentences2 = []
val_labels2 = [];

In [13]:
for i in 1:8534
    if train_labels[i] != 0.5
        push!(train_labels2, mapfloat(train_labels[i]))
        push!(train_sentences2, train_sentences[i])
    end
end

for i in 1:2210
    if test_labels[i] != "neu"
        push!(test_labels2, mapstring(test_labels[i]))
        push!(test_sentences2, test_sentences[i])
    end
end        

for i in 1:1100
    if val_labels[i] != "neu"
        push!(val_labels2, mapstring(val_labels[i]))
        push!(val_sentences2, val_sentences[i])
    end
end

In [14]:
length(train_sentences), length(test_sentences), length(val_sentences)

(8534, 2210, 1100)

In [15]:
length(train_sentences2), length(test_sentences2), length(val_sentences2)

(8328, 1821, 872)

In [16]:
train_sentences2 = vcat(train_sentences2, val_sentences2)
train_labels2 = vcat(train_labels2, val_labels2);
train_labels2 = hcat(train_labels2)[:,1]
test_labels2 = hcat(test_labels2)[:,1];

In [17]:
length(train_sentences2), length(test_sentences2), length(val_sentences2)

(9200, 1821, 872)

In [18]:
vocab = []
for sentence in all_sentences
    for word in sentence
        if !(word in vocab); push!(vocab, word);end
    end
end
push!(vocab, "UNK");

In [19]:
w2i = Dict()
i2w = Dict()
dictindex = 1
for word in vocab
    w2i[word] = dictindex
    i2w[dictindex] = word
    dictindex+=1
end
w2i["UNK"] = 19507
i2w[19507] = "UNK";

In [20]:
train_sentences2 = [(x -> w2i[x]).(s) for s in train_sentences2];
test_sentences2 = [(x -> w2i[x]).(s) for s in test_sentences2];
all_sentences = [(x -> w2i[x]).(s) for s in all_sentences];

In [21]:
function strings_to_indices(s)
    s = split(s)
    out = []
    for word in s
        word = lowercase(word)
        if !(word in vocab)
            word = w2i["UNK"]
        end
        push!(out, w2i[word])
    end
    hcat(out)
end

strings_to_indices("hey there")

2×1 Array{Int64,2}:
 1804
   54

In [22]:
maxlength = 56
for sentence in train_sentences2
    while length(sentence) != maxlength
        pushfirst!(sentence, w2i["UNK"])
    end
end

In [23]:
maxlength = 56
for sentence in test_sentences2
    while length(sentence) != maxlength
        pushfirst!(sentence,w2i["UNK"])
    end
end

In [24]:
embedmatrix = []
no_embeddings = []
count = 0
for word in vocab
    if word in keys(embeddingdict)
        wordvector = (x-> parse(Float32, x)).(split(lines[embeddingdict[word]])[2:301])
        count += 1
    else
        wordvector = xavier(Float32, 300)
        push!(no_embeddings, (word, wordvector))
    end
    push!(embedmatrix, hcat(wordvector))
end

In [25]:
embedmatrix = hcat(embedmatrix...);

In [26]:
size(embedmatrix)

(300, 19507)

In [27]:
println(count, " out of ", length(vocab), " words are in Stanford Glove Embeddings. The rest is initialized randomly.")

18593 out of 19507 words are in Stanford Glove Embeddings. The rest is initialized randomly.


In [39]:
#Hyperparameters of the Model
BATCHSIZE=5               # Number of instances in a minibatch
EMBEDSIZE=300             # Word embedding size
NUMHIDDEN=100             # Hidden layer size
MAXLEN=150                # maximum size of the word sequence, pad shorter sequences, truncate longer ones
VOCABSIZE=length(vocab)   # maximum vocabulary size, keep the most frequent 30K, map the rest to UNK token
NUMCLASS=2                # number of output classes
DROPOUT=0.5               # Dropout rate
LR=0.002                  # Learning rate
BETA_1=0.9                # Adam optimization parameter
BETA_2=0.999              # Adam optimization parameter
EPS=1e-08                 # Adam optimization parameter
MAXLENGTH = 56            # Used for padding

56

In [40]:
dtrn = minibatch(train_sentences2, train_labels2,BATCHSIZE;shuffle=true)
dtst = minibatch(test_sentences2, test_labels2 ,BATCHSIZE)
length(dtrn), length(dtst)

(1840, 364)

In [84]:
d1 = Knet.load("lstmn8609.jld2")

Dict{String,Any} with 6 entries:
  "lstm"   => LSTM(input=300,hidden=100)
  "Wh"     => P(Array{Float64,2}(1,100))
  "output" => P(KnetArray{Float32,2}(2,100))
  "embeds" => P(KnetArray{Float32,2}(300,19507))
  "Whh"    => P(Array{Float64,2}(1,100))
  "Wx"     => P(Array{Float64,2}(1,300))

In [85]:
best_lstm = d1["lstm"]
best_output = d1["output"]
best_embeds = d1["embeds"]
best_Wh = d1["Wh"]
best_Whh = d1["Whh"]
best_Wx = d1["Wx"];

In [86]:
best_Wh = KnetArray((x->Float32(x)).(value(best_Wh)))
best_Whh = KnetArray((x->Float32(x)).(value(best_Whh)))
best_Wx = KnetArray((x->Float32(x)).(value(best_Wx)))

1×300 KnetArray{Float32,2}:
 0.0499197  -0.00472626  0.0130879  …  -0.03708  0.0263653  -0.045572

In [102]:
#model struct
struct LSTMN
    embeds
    lstm1
    lstm2
    output
    pdrop
    Wh
    Wx
    Whh
    memory_tape
    hidden_tape
end

ErrorException: invalid redefinition of constant LSTMN

In [88]:
#model constructor
function LSTMN(input::Int, embed::Int, hidden::Int, output::Int; pdrop=0)
    #=
    embeds = param(KnetArray(embedmatrix))
    lstm = RNN(embed,hidden)
    output = param(output, hidden)
    Wh = param(KnetArray(xavier(Float32, 1,100)))
    Wx = param(KnetArray(xavier(Float32, 1,300)))
    Whh = param(KnetArray(xavier(Float32, 1,100)))
    =#
    
    embeds = best_embeds
    lstm = best_lstm
    output = best_output
    Wh = best_Wh
    Wx = best_Wx
    Whh = best_Whh
    
    memory_tape = KnetArray(zeros(Float32, 100,5,56))
    hidden_tape = KnetArray(zeros(Float32, 100,5,56))
    LSTMN(embeds, lstm, output, pdrop, Wh, Wx, Whh, memory_tape, hidden_tape)
end

LSTMN

In [99]:
function (lstmn::LSTMN)(input)
    embed = lstmn.embeds[:, permutedims(hcat(input...))]
    embed = dropout(embed,lstmn.pdrop)
    
    memory_tape = KnetArray(zeros(Float32,100,5,52))
    hidden_tape = KnetArray(zeros(Float32,100,5,52))
    
    hprev = zeros(100,5)
    xt = embed[:,:,1]
    
    #ht = lstmn.lstm.h[:,:,1]
    #ct = lstmn.lstm.c[:,:,1]

    ht = randn(100,5)
    
    for t in 2:52
        h = hidden_tape[:,:,1:t-1]
        c = memory_tape[:,:,1:t-1]
    
        dot1 = lstmn.Wh * reshape(h, 100, 5*(t-1))
        dot1 = reshape(dot1, (t-1), 5)
                 
        xt = embed[:,:,t]
        dot2 = lstmn.Wx * xt
        
        hprev = hidden_tape[:,:,t-1]
        dot3 = lstmn.Whh * hprev
        
        at = tanh.(dot1 .+ dot2 .+ dot3)        
        
        soft = softmax(at; dims=1)
        soft = permutedims(soft)
        soft = reshape(soft, 5*(t-1))
        
        h = reshape(h, 100, 5*(t-1))
        h = permutedims(h)
        
        c = reshape(c, 100, 5*(t-1))
        c = permutedims(c)
    
        new_h = soft .* h
        new_h = reshape(new_h, 5, (t-1), 100)
        new_h = sum(new_h; dims = 2)
        new_h = reshape(new_h, 5,100)

        new_c = soft .* c
        new_c = reshape(new_c, 5, (t-1), 100)
        new_c = sum(new_c; dims = 2)
        new_c = reshape(new_c, 5,100)
 
        #ht = lstmn.lstm(xt)
        ct = lstmn.lstm.c[:,:,1]
        
        hidden_tape[:,:,t] = ht
    end
    hidden = lstmn.lstm(embed)
    hidden = dropout(ht,lstmn.pdrop)
    return lstmn.output * hidden    
end

#model(input,output)
(l::LSTMN)(input,output) = nll(l(input),output)
#model(data)
(l::LSTMN)(d::Knet.Data) = Knet.mean(l(x,y) for (x,y) in d)

In [100]:
model = LSTMN(VOCABSIZE,EMBEDSIZE,NUMHIDDEN,NUMCLASS,pdrop=DROPOUT);

In [101]:
x1, y1 = first(dtrn)
model(x1,y1)

MethodError: MethodError: no method matching *(::KnetArray{Float32,2}, ::Array{Float64,2})
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502
  *(!Matched::LinearAlgebra.Adjoint{#s623,#s622} where #s622<:LinearAlgebra.AbstractQ where #s623, ::Union{DenseArray{T,1}, DenseArray{T,2}, ReinterpretArray{T,1,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray}, ReinterpretArray{T,2,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray}, ReshapedArray{T,1,A,MI} where MI<:Tuple{Vararg{SignedMultiplicativeInverse{Int64},N} where N} where A<:Union{ReinterpretArray{T,N,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{Abstract

In [92]:
@time accuracy(model,dtst)

MethodError: MethodError: no method matching getindex(::Int64, ::Colon, ::Colon, ::Int64)
Closest candidates are:
  getindex(::Number, !Matched::Integer...) at number.jl:82
  getindex(::Number) at number.jl:75
  getindex(::Number, !Matched::Integer) at number.jl:77
  ...

In [79]:
models = []
function fasttrain!(lstmn::LSTMN, dtrn, dtst, max_iters=500)
    a = adam(lstmn, take(cycle(dtrn), max_iters+1);lr=LR,beta1=BETA_1,beta2=BETA_2,eps=EPS)
    progress!(a)
    push!(models, lstmn)
end            

fasttrain! (generic function with 2 methods)

In [80]:
function trainuntil(lstmn::LSTMN, dtrn, dtst, until=0.85)
    acc = accuracy(model, dtst)
    while (acc < until)
        fasttrain!(lstmn, dtrn, dtst,1)
        #acc = accuracy(model, dtst)
        #println(acc)
    end
    acc
end

trainuntil (generic function with 2 methods)

In [81]:
function besttrain!(lstmn::LSTMN, dtrn, dtst, max_iter=100)
    best_acc = 0
    best_model = model
    acc = accuracy(model, dtst)
    i = 0
    while (i < max_iter)
        fasttrain!(lstmn, dtrn, dtst,10)
        acc = accuracy(model, dtst)
        if(acc > best_acc)
            best_model = model
            best_acc = acc
        end
        println("current accuracy:", acc, "best accuracy:", best_acc)
        i+=1
    end
    best_model
end

besttrain! (generic function with 2 methods)

In [196]:
accuracy(model, dtst), accuracy(model, dtrn)

(0.8153846153846154, 0.9992391304347826)

In [ ]:
besty1 = trainuntil(model, dtrn, dtst, 0.862)

5.94e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 108.52i/s]
1.83e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.73i/s]
1.43e-02  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.49i/s]
8.20e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 111.16i/s]
6.96e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.63i/s]
6.77e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.27i/s]
2.24e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.90i/s]
5.98e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.52i/s]
1.38e-03  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 111.58i/s]
1.52e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.09i/s]
3.86e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.28i/s]
1.53e-03  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.02i/s]
9.74e-04  100.00

2.35e-03  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.96i/s]
3.70e-04  100.00%┣███████████████████████████████████┫ 2/2 [00:02/00:02, 1.23i/s]
1.47e-03  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.41i/s]
1.02e-03  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.87i/s]
2.68e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.71i/s]
4.54e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 111.14i/s]
0.00e+00  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.90i/s]
2.43e-03  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.02i/s]
7.72e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.20i/s]
4.14e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.53i/s]
1.24e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.46i/s]
1.57e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.59i/s]
7.82e-06  100.00

2.79e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.40i/s]
7.37e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.77i/s]
7.27e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.90i/s]
6.29e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.35i/s]
4.67e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.99i/s]
8.88e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.58i/s]
1.53e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.47i/s]
3.29e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.84i/s]
7.92e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.50i/s]
3.62e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.69i/s]
1.12e-03  100.00%┣███████████████████████████████████┫ 2/2 [00:02/00:02, 1.23i/s]
2.77e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.11i/s]
6.77e-06  100.00

9.47e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.65i/s]
5.46e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.27i/s]
3.93e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.16i/s]
2.59e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.89i/s]
2.03e-02  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 111.20i/s]
9.63e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.15i/s]
5.63e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.74i/s]
1.34e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.64i/s]
4.77e-07  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.92i/s]
3.11e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.71i/s]
1.83e-01  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.00i/s]
5.91e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.84i/s]
6.68e-03  100.00

3.34e-06  100.00%┣███████████████████████████████████┫ 2/2 [00:02/00:02, 1.23i/s]
4.77e-07  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.40i/s]
2.07e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.62i/s]
2.03e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.97i/s]
5.05e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.34i/s]
8.87e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.12i/s]
8.20e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.08i/s]
6.05e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.34i/s]
3.32e-02  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.88i/s]
3.91e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.14i/s]
6.58e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.24i/s]
2.63e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.87i/s]
1.29e-04  100.00

...

8.58e-07  100.00%┣███████████████████████████████████┫ 2/2 [00:02/00:02, 1.22i/s]
1.00e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.82i/s]
2.16e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 111.11i/s]
1.89e-02  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.94i/s]
9.57e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.38i/s]
6.68e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.32i/s]
7.22e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 111.53i/s]
8.77e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.51i/s]
2.57e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.71i/s]
6.04e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.74i/s]
5.90e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 111.09i/s]
2.86e-07  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.36i/s]
2.02e-03  100.00

2.48e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.88i/s]
1.30e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.35i/s]
4.43e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.85i/s]
4.18e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.16i/s]
1.24e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.28i/s]
9.51e-03  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.48i/s]
1.34e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 111.31i/s]
8.59e-03  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.09i/s]
9.63e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.82i/s]
5.52e-05  100.00%┣███████████████████████████████████┫ 2/2 [00:02/00:02, 1.22i/s]
1.37e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.19i/s]
1.91e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.32i/s]
1.91e-06  100.00

2.86e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.12i/s]
1.00e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.70i/s]
1.57e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.41i/s]
7.96e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.09i/s]
3.05e-03  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.39i/s]
3.07e-01  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.90i/s]
7.57e-05  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 111.53i/s]
1.53e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.65i/s]
2.69e-03  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.20i/s]
7.83e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.94i/s]
1.43e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.83i/s]
3.72e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 109.36i/s]
1.23e-04  100.00

Excessive output truncated after 524311 bytes.

2.19e-06  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.86i/s]
2.03e-04  100.00%┣████████████████████████████████▉┫ 2/2 [00:00/00:00, 110.36i/s]
1.34e-06  100.00%┣

In [ ]:
accuracy(model, dtrn), accuracy(model, dtst)

In [59]:
Knet.save("lstmn869.jld2", "embeds", model.embeds, "lstm", model.lstm, "output", model.output, "Wh", model.Wh, "Wx", model.Wx, "Whh", model.Whh)